# Báo cáo bài tập lớn môn Học Máy

Họ và tên: Phú Minh Nhật

Mã sinh viên: 18020976

Lớp môn học: INT3405_1

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns

# Giới thiệu
**Mô tả vấn đề:**

Một trong những vấn đề hiện hữu trên các website ngày nay là làm cách nào để xử lý những nội dung độc hại và gây chia rẽ. Quora muốn đối mặt và giải quyết vấn đề này để giữ cho nền tảng của họ là một nơi an toàn để người dùng có thể chia sẻ kiến thức ra toàn cầu.

Quora là một nền tảng khuyến khích mọi người học hỏi lẫn nhau. Trên đó, mọi người có thể đặt câu hỏi và kết nối đến những người có thể đóng góp những câu trả lời chất lượng. Thách thức mấu chốt là loại bỏ các câu hỏi không thành thật - của những người hỏi dựa trên những thông tin sai lệch, hoặc nhằm đưa ra ý kiến cá nhân hơn là tìm kiếm câu trả lời.



**Mô tả dữ liệu:**

Dữ liệu để train bao gồm rất nhiều câu hỏi đã được hỏi và đã được đánh nhãn là insincere (target = 1) hoặc không (target = 0).



In [ ]:
#Đọc vào dữ liệu dùng để huấn luyện
data_raw = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
data_raw.head()

In [ ]:
insincere_questions = data_raw[data_raw['target'] == 1].question_text
sincere_questions = data_raw[data_raw['target'] == 0].question_text

In [ ]:
#Ví dụ về câu hỏi insincere
insincere_questions.sample(3, random_state=1).values

In [ ]:
#Ví dụ về câu hỏi sincere
sincere_questions.sample(3, random_state=1).values

# Phân tích và trực quan hóa dữ liệu

**Phân tích dữ liệu thô**

Có thể nhận thấy dữ liệu hiện tại đang thiếu cân bằng, khi mà hầu hết các câu hỏi đều được đánh nhãn sincere, và chỉ có một số lượng nhỏ là insincere

In [ ]:
# Đếm số lượng câu hỏi được đánh nhãn sincere và insincere
values = data_raw.target.value_counts()
print(values)

# Tính tỉ lệ giữa số lượng câu hỏi đánh nhãn sincere với câu hỏi đánh nhãn insincere
sincere_q_pc = values[0]/values.sum()*100
insincere_q_pc = values[1]/values.sum()*100
print('\n{}% of questions are sincere while {}% are insincere'.format(sincere_q_pc, insincere_q_pc))

In [ ]:
# Vẽ đồ thị miêu tả sự chênh lệch giữa số câu hỏi sincere và insincere
names = ['Sincere', 'Insincere']

plt.bar(names, values)
plt.suptitle('Number of Sincere and Insincere Questions')
plt.show()


In [ ]:
# Import thư viện wordcloud
from wordcloud import WordCloud, ImageColorGenerator

# Tách các câu hỏi thành các dictionary chứa các từ độc nhất và tần suất xuất hiện
def word_freq_dict(text):
    # Convert text into word list
    wordList = text.split()
    # Generate word freq dictionary
    wordFreqDict = {word: wordList.count(word) for word in wordList}
    return wordFreqDict

In [ ]:
# Vẽ wordcloud của dictionary chứa tần suất của từ
def word_cloud_from_frequency(word_freq_dict, title, figure_size=(10,6)):
    wordcloud.generate_from_frequencies(word_freq_dict)
    plt.figure(figsize=figure_size)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(title)
    plt.show()

In [ ]:
# Wordcloud của 1 tập 1000 câu hỏi insincere ngẫu nhiên
insincere_questions = data_raw.question_text[data_raw['target'] == 1]
insincere_sample = " ".join(insincere_questions.sample(1000, random_state=1).values)
insincere_word_freq = word_freq_dict(insincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Reds',
    background_color='white')
word_cloud_from_frequency(insincere_word_freq, "Các từ xuất hiện nhiều nhất trong tập 1000 câu hỏi đánh nhãn insincere") 

In [ ]:
# Wordcloud của tập 1000 câu hỏi sincere ngẫu nhiên
sincere_questions = data_raw.question_text[data_raw['target'] == 0]
sincere_sample = " ".join(sincere_questions.sample(1000, random_state=1).values)
sincere_word_freq = word_freq_dict(sincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Greens',
    background_color='white')

word_cloud_from_frequency(sincere_word_freq, "Các từ xuất hiện nhiều nhất trong tập 1000 câu hỏi chưa xử lý đánh nhãn sincere")

Có thể thấy, những từ xuất hiện nhiều nhất là những từ được sử dụng rất nhiều và thường xuyên như 'what', 'is', 'with', 'are',... nên không có tác dụng gì trong mô hình. Vậy nên, những từ phổ biến này (còn gọi là stopword) cần phải được loại bỏ.

In [ ]:
import nltk
import sys
import spacy

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string

Để chuẩn hóa văn bản và giảm thiểu độ nhiễu, quá trình tiền xử lý được áp dụng theo các bước sau:

*   Bước 1: Chuyển tất cả các ký tự về ký tự in thường
*   Bước 2: Chia các văn bản thành các list
*   Bước 3: Loại bỏ toàn bộ các dấu câu
*   Bước 4: Xóa tất cả các stopword có trong câu
*   Bước 5: Stemming - một kỹ thuật trích xuất từ cơ sở bằng cách loại bỏ các phụ tố của từ

Công cụ được sử dụng để thực hiện tiền xử lý là NTLK (Natural Language Toolkit)




In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])
stop = set(stopwords.words('english'))
punc = set(string.punctuation)

def clean_text(text):
    # Chuyển toàn bộ văn bản sang chữ in thường
    text = text.lower()
    # Tách câu văn thành list các từ
    wordList = text.split()
    # Loại bỏ các dấu câu
    wordList = ["".join(x for x in word if (x=="'")|(x not in punc)) for word in wordList]
    # Loại bỏ stop words
    wordList = [word for word in wordList if word not in stop]
    # Stem
    porter = PorterStemmer()
    wordList = [porter.stem(word) for word in wordList]

    reformed_sentence = " ".join(wordList)
    doc = nlp(reformed_sentence)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
question = data_raw.question_text.sample(1, random_state=1).values[0]
question

In [ ]:
# Kiểm tra xem hàm có hoạt động hay không
clean_text(question)

In [ ]:
# Thực hiện clean toàn bộ dữ liệu
data_raw['clean_text'] = data_raw['question_text'].astype('str').apply(clean_text)

In [ ]:
data_raw.clean_text.head()

In [ ]:
# Wordcloud của tập 1000 câu hỏi insincere ngẫu nhiên đã clean
clean_insincere_questions = data_raw.clean_text[data_raw['target'] == 1]
clean_insincere_sample = " ".join(clean_insincere_questions.sample(1000, random_state=1).values)
clean_insincere_word_freq = word_freq_dict(clean_insincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Reds',
    background_color='white')

word_cloud_from_frequency(clean_insincere_word_freq, "Các từ xuất hiện nhiều nhất trong tập 1000 câu hỏi đã làm sạch đánh nhãn insincere") 

In [ ]:
# Wordcloud của tập 1000 câu hỏi sincere ngẫu nhiên đã clean
clean_sincere_questions = data_raw.clean_text[data_raw['target'] == 0]
clean_sincere_sample = " ".join(clean_sincere_questions.sample(1000, random_state=1).values)
clean_sincere_word_freq = word_freq_dict(clean_sincere_sample)
wordcloud = WordCloud(width= 5000,
    height=3000,
    max_words=200,
    colormap='Greens',
    background_color='white')

word_cloud_from_frequency(clean_sincere_word_freq, "Các từ xuất hiện nhiều nhất trong tập 1000 câu hỏi đã làm sạch đánh nhãn sincere") 

# Text2Vec
Cần phải biến đổi dữ liệu từ dạng text sang dạng vector

**Bag Of Word**

BOW là một mô hình cơ bản được dùng trong tác vụ xử lý ngôn ngữ tự nhiên. Nó được gọi như vậy vì mô hình này bỏ đi thứ tự của từ trong văn bản, chỉ thể hiện sự xuất hiện và tần suất xuất hiện của từ trong văn bản.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_converter = CountVectorizer()

In [ ]:
sample_question_text = data_raw['clean_text'].sample(1, random_state= 1).values
sample_question_text

In [ ]:
sample_count_vectorized_data = bow_converter.fit_transform(sample_question_text)
sample_count_vectorized_data.toarray()

In [ ]:
count_vectorized_data_feature_names = bow_converter.get_feature_names()
count_vectorized_data_feature_names

# TF-IDF
TF-IDF (viết tắt của term frequency – inverse document frequency) là một phương thức thống kê thường được sử dụng trong mảng truy xuất thông tin (information retrieval) và khai phá dữ liệu văn bản (text mining) để đánh giá mức độ quan trọng của một cụm từ đối với một tài liệu cụ thể trong một tập hợp bao gồm nhiều tài liệu. Phương thức này bao gồm 2 khái niệm:

+ TF (Term Frequency - Tần suất xuất hiện của từ) là số lần từ xuất hiện trong văn bản. Vì các văn bản có thể có độ dài ngắn khác nhau nên một số từ có thể xuất hiện nhiều lần trong một văn bản dài hơn là một văn bản ngắn. Như vậy, term frequency thường được chia cho độ dài văn bản( tổng số từ trong một văn bản).

+ IDF (Inverse Document Frequency - Nghịch đảo tần suất của văn bản), giúp đánh giá tầm quan trọng của một từ . Khi tính toán TF , tất cả các từ được coi như có độ quan trọng bằng nhau. Nhưng  một số từ như “is”, “of” và “that” thường xuất hiện rất nhiều lần nhưng độ quan trọng là không cao. Như thế chúng ta cần giảm độ quan trọng của những từ này xuống.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_converter = TfidfVectorizer(ngram_range=(1,1))

In [ ]:
sample_tfidf_vectorized_data = tfidf_converter.fit_transform(sample_question_text)
sample_tfidf_vectorized_data.toarray()

In [ ]:
tfidf_word_feature_names = tfidf_converter.get_feature_names()
tfidf_word_feature_names

In [ ]:
len(tfidf_word_feature_names)

# Mô hình
Mô hình trong bài toán này sẽ sử dụng thuật toán Hồi quy Logistics (Logistics Regression). Tìm hiểu sâu hơn về LR:

https://excessive-source-1c9.notion.site/16-09-2021-H-i-quy-Logistics-cdcc911147e5458ba9203b58e6bd0099


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

count_vectorizer = CountVectorizer()
model = LogisticRegression(C=1, random_state=0, max_iter=1000)

vectorize_logit_pipeline = Pipeline([
    ('count_vectorizer', count_vectorizer),
    ('logit', model)
])

# Pipeline với LR và Count Vectorizer


In [ ]:
# Biến đầu vào
X = data_raw['clean_text']
# Biến đầu ra
y = data_raw['target']

Chia bộ dữ liệu huấn luyện thành 2 tập, với 1 tập để huẩn luyện, và 1 tập để kiểm thử

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

Huấn luyện mô hình sử dụng tập dữ liệu huấn luyện và đặc trưng

In [ ]:
vectorize_logit_pipeline.fit(train_X, train_y)

Đưa ra dự đoán của mô hình

In [ ]:
predictions = vectorize_logit_pipeline.predict(test_X)

Kiểm tra điểm chính xác

In [ ]:
accuracy_score(test_y, predictions)

Kiểm tra điểm F1

In [ ]:
from sklearn.metrics import f1_score
f1_score(test_y, predictions)

Vẽ ma trận confusion

In [ ]:
confusion_matrix_logit_tfidf = confusion_matrix(test_y, predictions)
sns.heatmap(confusion_matrix_logit_tfidf, annot= True, xticklabels=['sincere', 'insincere'], yticklabels=['sincere', 'insincere'])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, predictions))

# Pipeline sử dụng LR và TF-IDF Bi-gram Vectorizer

In [ ]:
tfidf_ngrams_converter = TfidfVectorizer(ngram_range=(1,2))
tfidf_ngrams_logit_pipeline = Pipeline([
    ('tfidf_vectorizer', tfidf_ngrams_converter),
    ('logit', model)
])

In [ ]:
tfidf_ngrams_logit_pipeline.fit(train_X, train_y)

In [ ]:
new_predictions = tfidf_ngrams_logit_pipeline.predict(test_X)

In [ ]:
accuracy_score(test_y, new_predictions)

In [ ]:
f1_score(test_y, new_predictions)

In [ ]:
confusion_matrix_logit_tfidf = confusion_matrix(test_y, new_predictions)
sns.heatmap(confusion_matrix_logit_tfidf, annot= True, xticklabels=['sincere', 'insincere'], yticklabels=['sincere', 'insincere'])

In [ ]:
print(classification_report(test_y, new_predictions, target_names=['sincere', 'insincere']))

Có thể thấy, mô hình khi áp dụng các vector đặc trưng Bi-gram cho điểm chính xác và điểm F1 cao hơn

In [ ]:
test_data = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
test_data.head()

In [ ]:
test_data.info()

In [ ]:
test_data['clean_text'] = test_data['question_text'].astype('str').apply(clean_text)

In [ ]:
test_data.head()

In [ ]:
x_final = test_data['clean_text']

In [ ]:
y_final = tfidf_ngrams_logit_pipeline.predict(x_final)

In [ ]:
test_data['target'] = y_final

In [ ]:
result_df = test_data[['qid','target']]

In [ ]:
result_df.rename(columns={'target': 'prediction'}, inplace=True)
result_df.set_index('qid', inplace=True)
result_df.head()

In [ ]:
result_df.to_csv('submission.csv')
!head submission.csv